In [37]:
import numpy as np
import cv2 as cv
import json

"""缩小图像,方便看效果
resize会损失像素，造成边缘像素模糊，不要再用于计算的原图上使用
""" 
def resizeImg(src):
    height, width = src.shape[:2]
    size = (int(width * 0.3), int(height * 0.3))  
    img = cv.resize(src, size, interpolation=cv.INTER_AREA)
    return img

"""找出ROI，用于分割原图
原图有四块区域，一个是地块区域，一个是颜色示例区域，一个距离标尺区域，一个南北方向区域
理论上倒排后的最大轮廓的是地块区域
"""
def findROIContours(src):
    copy = src.copy()
    gray = cv.cvtColor(copy, cv.COLOR_BGR2GRAY)
    # cv.imshow("gray", gray)
    
    # 低于thresh都变为黑色，maxval是给binary用的
    # 白底 254, 255 黑底 0, 255
    threshold = cv.threshold(gray, 0, 255, cv.THRESH_BINARY)[1]
    # cv.imshow("threshold", threshold)
    contours, hierarchy = cv.findContours(threshold, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    sortedCnts = sorted(contours, key = cv.contourArea, reverse=True)
    # cv.drawContours(copy, [maxCnt], -1, (255, 0, 0), 2)
    # cv.imshow("roi contours", copy)
    return sortedCnts
    


"""按照mask，截取roi
"""
def getROIByContour(src, cnt):
    copy = src.copy()
    # black background
    mask = np.zeros(copy.shape[:2], np.uint8)
    mask = cv.fillConvexPoly(mask, cnt, (255,255,255)) 
#     cv.imshow("mask", resizeImg(mask))
    # print(mask.shape)
    # print(copy.dtype)
    roi = cv.bitwise_and(copy, copy, mask=mask)
    # cv.imshow("roi", roi)
    
    # white background for none roi and fill the roi's backgroud
    mask = cv.bitwise_not(mask)
    whiteBg = np.full(copy.shape[:2], 255, dtype=np.uint8)
    whiteBg = cv.bitwise_and(whiteBg, whiteBg, mask=mask)
    whiteBg = cv.merge((whiteBg,whiteBg,whiteBg))
#     cv.imshow("whiteBg", resizeImg(whiteBg))
    
    roiWithAllWhite = cv.bitwise_or(roi, whiteBg)
    return roiWithAllWhite


"""找出所有的地块轮廓
"""
def findAllBlockContours(src):
    copy = src.copy()
    contours = findExternalContours(copy)
    return contours


"""找出颜色示例里的颜色BGR
根据限定长，高，比例来过滤出实例颜色区域
"""
def findBGRColors(cnts):
    W_RANGE = [170,180]
    H_RANGE = [75, 85]
    RATIO_RANGE = [0.40, 0.50]
    colors = []
    
    # TODO 如果可以知道颜色示例的个数可以提前统计退出循环
    for cnt in cnts:
        x,y,w,h = cv.boundingRect(cnt)
        ratio = round(h/w, 2)
        
        if ratio > RATIO_RANGE[0] and ratio < RATIO_RANGE[1] \
            and w > W_RANGE[0] and w < W_RANGE[1] \
            and h > H_RANGE[0] and h < H_RANGE[1]:
            # print(ratio,x,y,w,h)
            # 因为，原图色块矩形的周边和mask出来的颜色区都有模糊渐变的线
            # 无法使用cv.mean(colorRegion, meanMask)来计算实际颜色
            # 所以，取矩形的中心点的颜色最为准确
            cx,cy = round(x+w/2), round(y+h/2)
            bgr = img_white_bg[cy, cx]
            # print(bgr)
            colors.append(bgr)
    
    return colors


def drawnForTest(img, contours, rect=False):
    img = img.copy()
    for i in contours:
        if rect:
            x, y, w, h = cv.boundingRect(i)
            cv.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv.putText(img, 'Area' + str(cv.contourArea(i)), (x+5,y+15), cv.FONT_HERSHEY_PLAIN, 1,(255,0,0), 1, cv.LINE_AA)
        else:
            cv.drawContours(img, [i], -1, (0, 255, 0), 2)  
    cv.imshow("detect", resizeImg(img))
    cv.waitKey(0)


"""Find Original Contours
Find Original Contours from source image, we only need external contour.
Args:
    src: source image
Returns:
    Original contours
"""
def findExternalContours(src):
    # 必须是白色背景的图片，如果是黑色背景，将黑色改成白色
#     src[np.where((src == [0,0,0]).all(axis = 2))] = [255,255,255]
    
    # preprocess, remove noise, a lot noise on the road
    gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
    # 测试这里道理有没有必要高斯
#     blur = cv.GaussianBlur(gray, (3,3), 0)
    
    thresVal = 254
    maxVal = 255
    ret,thresh1 = cv.threshold(gray, thresVal, maxVal, cv.THRESH_BINARY)
    
    kernel = np.ones((7,7),np.uint8)
    morph = cv.morphologyEx(thresh1, cv.MORPH_CLOSE, kernel)
    
    # ??threshold怎么计算的？
    edges = cv.Canny(morph,100,200)
    # edges找出来，但是是锯齿状，会在找轮廓时形成很多点，这里加一道拉普拉斯锐化一下
    edges = cv.Laplacian(edges, -1, (3,3))
    
    contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
#     contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
#     cv.imshow('gray', resizeImg(gray))
#     cv.imshow('thresh1', resizeImg(thresh1))
#     cv.imshow('edges', resizeImg(edges))
#     cv.imshow('opening', opening)
#     cv.imshow('opening', resizeImg(opening))
    return contours, hierarchy        


"""根据找出的轮廓和层级关系计算地块和色块的父子关系
"""
def getBlockColorTree(copy, blockCnts, hierarchy):
    # print(hierarchy)
    # hierarchy [Next, Previous, First_Child, Parent]
    currentRootIndex = -1
    rootRegions = {}
    for i,cnt in enumerate(blockCnts):
        x,y,w,h = cv.boundingRect(cnt)
        cntArea = cv.contourArea(cnt)
        if cntArea > 1000:
            continue
        cv.putText(copy, str(i), (x+5,y+15), cv.FONT_HERSHEY_PLAIN, 1,(255,0,0), 1, cv.LINE_AA)

        print(i, hierarchy[0][i])
        if hierarchy[0][i][3] == -1:
            # root region
            currentRootIndex = i
            if currentRootIndex == len(blockCnts):
                break
            rootRegion = {'index': i, 'contour': cv.contourArea(blockCnts[currentRootIndex]), 'childRegion': []}
            rootRegions[currentRootIndex] = rootRegion
        elif hierarchy[0][i][3] == currentRootIndex:
            rootRegions[currentRootIndex]['childRegion'].append({'index': i, 'contour': cv.contourArea(cnt)})

    cv.imshow("blockCnts with debug info", resizeImg(copy))

    print(rootRegions)        
    data2 = json.dumps(rootRegions, sort_keys=True, indent=4, separators=(',', ': '))
    print(data2)

    
"""使用颜色来分块，并返回所有地块和色块父子关系
debug 只演示前三个地块的识别过程，可以通过debugFrom:debugLen来调整debug开始位置和长度
"""
def findColorRegionsForAllBlocks(img_white_bg, blockCnts, debug=False, debugFrom=0, debugLen=3):
    filteredBlockCnts = [cnt for cnt in blockCnts if cv.contourArea(cnt) > 100]
    
    if debug:
        filteredBlockCnts = filteredBlockCnts[debugFrom:debugLen]
    
    for blockCnt in filteredBlockCnts:
        findColorRegionsForBlock(img_white_bg, blockCnt, debug)

"""根据threshold重新计算BGR的值
"""
def bgrWithThreshold(bgr, threshold):        
    newBgr = []
    for x in bgr.tolist():
        if x + threshold < 0:
            newBgr.append(0)
        elif x + threshold > 255:
            newBgr.append(255)
        else:
            newBgr.append(x + threshold )
    
    return newBgr   
        
        
"""使用颜色来找出单个地块内的色块
"""
def findColorRegionsForBlock(img_white_bg, blockCnt, debug=False):  
    blockWithColorsDict = {'area': cv.contourArea(blockCnt), 'points':[] , 'children': []}
    
    blockRegion = getROIByContour(img_white_bg, blockCnt) 
    if debug:
        cv.imshow("blockRegions", np.hstack([resizeImg(img_white_bg), resizeImg(blockRegion)]))
        
    colorCnts = []
    for bgr in bgrColors:
        # 图片里的颜色可能和示例颜色不相等，适当增加点阈值来防色差
        threshold  = 5
        lower = np.array(bgrWithThreshold(bgr, -threshold), dtype="uint8")  
        upper = np.array(bgrWithThreshold(bgr, threshold), dtype="uint8")  
        # 根据阈值找到对应颜色区域，黑底白块
        mask = cv.inRange(blockRegion, lower, upper) 
        # cv.imshow("mask", resizeImg(mask))
        
        # 过滤出于颜色匹配的色块
        nonZeroCount = cv.countNonZero(mask)
        # print('none zero count', nonZeroCount)
        if nonZeroCount == 0:
            continue
            
        contours, hierarchy = cv.findContours(mask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
#         print('external', len(contours))
#         print(hierarchy)
        colorCnts.extend(contours)
        
        
#         contours, hierarchy = cv.findContours(mask.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
#         print('tree', len(contours))
#         print(hierarchy)

        if debug:
            # 黑白变白黑
            mask_inv = 255 - mask 
            # cv.imshow("mask_inv", resizeImg(mask_inv))
            # 展示图片
            output = cv.bitwise_and(blockRegion, blockRegion, mask=mask_inv) 
            # cv.drawContours(output, contours, -1, (0, 0, 255), 3)  
            
            cv.imshow("images", np.hstack([resizeImg(blockRegion), resizeImg(output)]))
            cv.waitKey(0)
        
        # 色块在同一个地块内也可能是多个的，色块内又内嵌了色块
        # TODO 嵌套的
        # colorCnts 循环递归，不到叶子节点，所有的色块统一当作地块继续处理，这样就可以解决嵌套问题
        # 所以要先构建一个三层数据模型
        # 第一层 是一个列表 存放所有的地块
        # 第二层 是以色块为节点，色块内无嵌套，则为叶子节点，有嵌套，色块升级为地块，继续深入查找色块，直到没有内嵌，成为叶子节点
        # 第三层 是以色块为叶子节点
        colorDicts = []
        for colorCnt in contours:
            colorDict = {'area': cv.contourArea(colorCnt), 'points':[], 'color': bgr.tolist()}
            colorDicts.append(colorDict)
        blockWithColorsDict['children'].extend(colorDicts)
        
    # print(blockWithColorsDict)        
    jsonData = json.dumps(blockWithColorsDict, sort_keys=True, indent=4, separators=(',', ': '))
    print(jsonData)    
    
    return colorCnts

    
# 用于找ROI
img = cv.imread('data_hierarchy2.png')
# 用于真实计算，
# 1. 色调区间误差，原图4识别最高，图3识别一般，需要增加threshold到5，
# 2. 间隙与边框误差，色块面积总和与地块面积相差3000左右，应该是线的面积没计算进去
# 3. 
img_white_bg = cv.imread('data_hierarchy4.png')

# 将图片按照轮廓排序，最大的是总地块
# 按照原图中的比例和实际距离来分割图片，参考findBGRColors的计算方式
sortedCnts = findROIContours(img)    
# print(len(sortedCnts[2:]))
# drawnForTest(img_white_bg, sortedCnts[3], rect=True)
# print(sortedCnts[3])
print(cv.boundingRect(sortedCnts[3]))
print(img_white_bg.shape)

# 2401 * 3151
# 670
px_km_scale = 670/1000
area_px = (2401*3151)
area_km2 = (2401*px_km_scale*3151*px_km_scale)
print(area_px/area_km2)
print(1/(px_km_scale*px_km_scale))

# 获取总地块
rootRegion = getROIByContour(img_white_bg, sortedCnts[0])
# cv.imshow("rootRegion", resizeImg(rootRegion))

# 找出色块的颜色
bgrColors = findBGRColors(sortedCnts[1:])
# print(bgrColors)


# 找出地块
copy = rootRegion.copy()
blockCnts, hierarchy = findAllBlockContours(copy)
# print(len(blockCnts))
# drawnForTest(img_white_bg, blockCnts, rect=False)

# 通过颜色来检测地块内色块
# findColorRegionsForAllBlocks(img_white_bg, blockCnts, debug=True, debugLen=1)
# findColorRegionsForAllBlocks(img_white_bg, blockCnts)


# 根据轮廓找地块的方法首先hierarchy的转换父子关系，还有很多小的干扰项待解决
# getBlockColorTree(copy, blockCnts, hierarchy)

cv.waitKey(0)
cv.destroyAllWindows()


(2238, 2132, 670, 80)
(2401, 3151, 3)
2.227667631989307
2.227667631989307


In [23]:
a = [1]
b = [2,3]
a.extend(b)
a

c=1
-c

-1